<div style="padding: 10px; margin-bottom: 10px; border: 1px solid #333333;">
    
# support mission   
    
### このsupport mission では、先ほど作成したmodelに基づき、カメラに映っている状況がfreeである確率とblockedである確率を返してくれる事を確認する

## <span style="color: red; ">NEW</span> カテゴリ：[モデルによる判定結果にもとづくロボットカーの制御]

## 完了の定義：jetbotの前に赤いレゴブロックで作られた壁が存在する場合、jetbotが停止し、そうでない場合は直進すること

</div>



このノートブックでは、トレーニングしたモデルを使用して、jetbotの衝突回避動作を有効にするために、jetbotがfreeであるかblockedな状況かを検出します。 

## 訓練されたモデルを読み込もう

トレーニングノートの指示に従って、best_model.pthをこのファイルと同じディレクトリに置いていることを前提としています。 


In [ ]:
import torch
import torchvision

# 学習済みモデルを初期化しています
model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

次に ``best_model.pth`` ファイルを読み込みます

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

現在、モデルはCPUメモリにあり、以下のコードを実行してGPUデバイスに転送します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)

## 前処理を行う関数を定義する
モデルを読み込みましたが、少し問題があります。 モデルをトレーニングした形式は、カメラの形式と完全には一致しません。 そのためには、いくつかの前処理を行う必要があり以下の手順をふむ必要があります

1. BGRからRGBに変換する
2. HWCレイアウトからCHWレイアウトに変換する
3. トレーニング中に行ったのと同じパラメーターを使用して正規化します（カメラは[0、255]の範囲の値を提供し、ロードされた画像は[0、1]の範囲でトレーニングします。したがって、255.0
4. CPUメモリからGPUメモリにデータを転送します
5. バッチディメンションを追加する

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

# カメラからの画像を正規化するメソッドを定義しています
def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

これで、カメラの形式からニューラルネットワークの入力形式に画像を変換できる前処理関数を定義しました。

それでは、カメラを起動して表示しましょう。 
jetbotがblockedであると判断する確率を表示するスライダーを作成します

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)
image = widgets.Image(format='jpeg', width=224, height=224)
# blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

# display(widgets.HBox([image, blocked_slider]))

次に、カメラの値が変更されるたびに呼び出される関数を作成します。 この関数は次の手順を実行します

1. カメラ画像の前処理
2. ニューラルネットワークを実行する
3. 出力に応じて動作を変更する

<div style="padding: 10px; margin-bottom: 10px; border: 1px solid #333333;">


## <span style="color: red; ">NEW</span> カテゴリ：[モデルによる判定結果にもとづくロボットカーの制御]



</div>

In [1]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    ## 削除した部分を下記のようにロボットを制御するためのプログラムに変更しました
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)
    ####################################################################
update({'new': camera.value})  # we call the function once to intialize

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-1-5ec97c371413>", line 16, in <module>
    update({'new': camera.value})  # we call the function once to intialize
NameError: name 'camera' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/usr/local/lib

NameError: name 'camera' is not defined

In [ ]:
camera.observe(update, names='value')

In [ ]:
camera.unobserve(update, names='value')
robot.stop()

In [ ]:
camera_link.unlink()

In [ ]:
camera_link.link()